In [1]:
import gym
import numpy as np

from gym.spaces import Box

from metadrive.envs.metadrive_env import MetaDriveEnv
from clip.clip_rewarded_ppo import CLIPRewardedPPO
from config import CONFIGS

CONFIG = CONFIGS["1"]
w, h = (100, 100)

/opt/anaconda3/envs/metadrive/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from custom import SegCamera, EgoState, WaypointSensor, AgentObservation

sensors = dict(
    seg_camera=(SegCamera, w, h),
    ego=(EgoState, 'default_agent'),
    waypoint=(WaypointSensor, 'default_agent', 15),
)

env_cfg = dict(
    agent_observation=AgentObservation,

    # Image rendering
    image_observation=True,
    norm_pixel=False,
    stack_size=1,

    # Sensors
    sensors=sensors
)

### Extra just for dimension testing

In [3]:
import matplotlib.pyplot as plt

env = MetaDriveEnv(env_cfg)

try:
    obs, info = env.reset()
    agent = env.agent
    obs, reward, terminated, truncated, info = env.step(env.action_space.sample())
finally:
    env.close()

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector(), seg_camera: SegCamera(100, 100), ego: EgoState('default_agent',), waypoint: WaypointSensor('default_agent', 15)]
[INFO] Render Mode: offscreen
[INFO] Horizon (Max steps per agent): 1000
[WARNING] You have set norm_pixel = False, which means the observation will be uint8 values in [0, 255]. Please make sure you have parsed them later before feeding them to network! (metadrive_env.py:117)
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1
